# Creating and simulating custom robot arms

In [1]:
import numpy as np
import sympy as sp
from packages.robot_arm import RobotArm
from packages.pybullet_sim import PybulletSimulation

### Calculate DH matrix and jacobian of the robot 

In [2]:
d1, theta2, theta3, d4, l2, l3 = sp.symbols('d1, theta2, theta3, d4, l2, l3')
robot = RobotArm()
robot.add_prismatic_joint(0, d1, 0, 0)
robot.add_revolute_joint(theta2, 0, l2, 0)
robot.add_revolute_joint(theta3, 0, l3, sp.pi)
robot.add_prismatic_joint(0, d4, 0, 0)
robot.add_subs([(l2, 2), (l3, 2)])
robot.load_robot()

In [3]:
robot.get_dh_matrix()

Matrix([
[cos(theta2 + theta3),  sin(theta2 + theta3),  0, l2*cos(theta2) + l3*cos(theta2 + theta3)],
[sin(theta2 + theta3), -cos(theta2 + theta3),  0, l2*sin(theta2) + l3*sin(theta2 + theta3)],
[                   0,                     0, -1,                                  d1 - d4],
[                   0,                     0,  0,                                        1]])

In [4]:
robot.jacobian()

Matrix([
[0, -l2*sin(theta2) - l3*sin(theta2 + theta3), -l3*sin(theta2 + theta3),  0],
[0,  l2*cos(theta2) + l3*cos(theta2 + theta3),  l3*cos(theta2 + theta3),  0],
[1,                                         0,                        0, -1],
[0,                                         0,                        0,  0],
[0,                                         0,                        0,  0],
[0,                                         1,                        1,  0]])

### 

### Interact with the robot with prismatic gripper

In [7]:
theta1, theta2, theta4, d1, d2, d3, a1, a2 = sp.symbols('theta1, theta2, theta4, d1, d2, d3, a1, a2')
robot = RobotArm()
robot.add_revolute_joint(theta1, d1, a1, 0)
robot.add_revolute_joint(theta2, 0, a2, -sp.pi)
robot.add_prismatic_joint(0, d3, 0, 0)
robot.add_revolute_joint(theta4, 0, 0, 0)
robot.add_subs([(d1, 3),(a1, 3),(a2, 3)])
robot.add_attachment()
robot.interact('inverse')

### Interact with the robot with revolute gripper

In [6]:
theta1, d2, d3, theta4, theta5, theta6 = sp.symbols('theta1, d2, d3, theta4, theta5, theta6')
robot = RobotArm()
robot.add_revolute_joint(theta1, 1, 0, 0)
robot.add_prismatic_joint(0, d2, 0, -sp.pi/2)
robot.add_prismatic_joint(0, d3, 0, 0)
robot.add_revolute_joint(theta4, 0.1, 0, sp.pi/2)
robot.add_revolute_joint(theta5, 0, 0.1, -sp.pi/2)
robot.add_revolute_joint(theta6, 0, 2, 0,)
robot.add_attachment('revolute_gripper',orientation = (0,-90,0))
robot.attachment_open_targets = (0.548,0.548)
robot.attachment_close_targets = (0,0)
robot.interact()

### Import existing robot and replay saved logs

In [8]:
robot = RobotArm()
robot.import_robot('robot_arms/my_robot/my_robot.urdf')
robot.load_robot()
robot.replay_logs('inverse')

### Example for detecting an object and picking it up with the arm

In [12]:
sim = PybulletSimulation()
sim.connect()
sim.load_table()
# place a lego randomly on the table
sim.load_lego((np.random.uniform(-1.5,1.5), np.random.uniform(-1.5,1.5),1),scaling=10) 
robot = RobotArm((-3,0,0.4),joint_forces=400)
robot.import_robot('robot_arms/my_robot/my_robot.urdf')
# set the dynamic condition to have grater precision
robot.set_dynamic_conditions(1000,0.001) 
robot.load_robot()

#### Steps to find and take the object:
* Move to the center of the table
* Take the picture from the end effector
* Find the average coordinates for the lego with the segmentation image
* Convert back to world coordinates 
* Bring that point up by 0.75 to better place the gripper
* Move to the target and pick up the object

In [13]:
robot.move2point((0,0,2.5),(180,0,0))
camera = robot.capture_image(fov=80,capture_now=True)
h,w = np.mean((np.where(camera.seg == 1)), axis=1)
h,w = round(h),round(w)
target_position = camera.rgbd_2_world(w,h,camera.depth[h,w]) +[0,0,0.75]
robot.move2point(target_position,(180,0,0))
robot.actuate_attachment(joint_targets = robot.attachment_close_targets)
robot.move2point((0,0,2),(180,0,0))

True

In [14]:
robot.quit_simulation(True)

True